# Loan Approval Prediction - Machine Learning Project

## Complete ML Project Analysis

This project analyzes the loan_data dataset to predict loan approval status based on various applicant characteristics.

## Table of Contents
1. [Introduction](#introduction)
2. [Dataset Overview](#dataset-overview)
3. [Exploratory Data Analysis](#eda)
4. [Data Preprocessing](#preprocessing)
5. [Model Building](#model-building)
6. [Model Evaluation](#evaluation)
7. [Insights & Conclusions](#insights-conclusions)
8. [Summary](#summary)

## Introduction <a id='introduction'></a>

This machine learning project aims to predict loan approval status (loan_status) based on various factors related to loan applicants. The dataset contains personal and financial information of loan applicants, allowing us to identify key factors that influence loan approval decisions.

### Objectives:
- Analyze the loan dataset to understand patterns in loan approvals
- Build predictive models to forecast loan approval likelihood
- Identify key factors influencing loan approval decisions
- Evaluate model performance and provide actionable insights

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

## Dataset Overview <a id='dataset-overview'></a>

Let's load the dataset and understand its structure, dimensions, and basic statistics.

In [ ]:
# Load the dataset
df = pd.read_csv('loan_data.csv')
print(f"Dataset Shape: {df.shape}")
print(f"\nColumn Names:")
for col in df.columns:
    print(f"- {col}")

print(f"\nDataset Info:")
df.info()

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Basic statistics
df.describe().T

### Target Variable Distribution

Understanding the distribution of our target variable (loan_status) is crucial for model building.

In [ ]:
# Check target variable distribution
target_dist = df['loan_status'].value_counts()
print("Target Variable Distribution:")
print(target_dist)
print(f"\nClass Imbalance: {target_dist[0]} negative cases ({target_dist[0]/len(df)*100:.2f}%) and {target_dist[1]} positive cases ({target_dist[1]/len(df)*100:.2f}%)")

# Visualize target distribution
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.pie(target_dist.values, labels=['Rejected (0)', 'Approved (1)'], autopct='%1.1f%%', startangle=90)
plt.title('Loan Status Distribution')

plt.subplot(1, 2, 2)
sns.countplot(data=df, x='loan_status')
plt.title('Loan Status Count')
plt.xlabel('Loan Status (0=Rejected, 1=Approved)')
plt.ylabel('Count')
plt.show()

## Exploratory Data Analysis (EDA) <a id='eda'></a>

Let's explore the relationships between different features and the target variable.

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
if missing_values.sum() == 0:
    print("No missing values found in the dataset.")
else:
    print("Missing values per column:")
    print(missing_values[missing_values > 0])

In [ ]:
# Separate numerical and categorical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"Numerical columns: {numerical_cols}")
print(f"Categorical columns: {categorical_cols}")

In [ ]:
# Distribution of numerical features
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.ravel()

for i, col in enumerate(numerical_cols):
    if i < len(axes):
        axes[i].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
        axes[i].set_title(f'Distribution of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')
        
# Hide unused subplots
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Box plots for numerical features by loan status
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.ravel()

for i, col in enumerate(numerical_cols):
    if i < len(axes) and col != 'loan_status':
        sns.boxplot(data=df, x='loan_status', y=col, ax=axes[i])
        axes[i].set_title(f'{col} by Loan Status')
        
# Hide unused subplots
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Distribution of categorical features
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.ravel()

for i, col in enumerate(categorical_cols):
    if i < len(axes):
        # Count plot for categorical features
        sns.countplot(data=df, x=col, ax=axes[i])
        axes[i].set_title(f'Distribution of {col}')
        axes[i].tick_params(axis='x', rotation=45)
        
# Hide unused subplots
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Categorical features vs loan status
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.ravel()

for i, col in enumerate(categorical_cols):
    if i < len(axes):
        # Stacked bar chart showing loan status by category
        crosstab = pd.crosstab(df[col], df['loan_status'], normalize='index')
        crosstab.plot(kind='bar', stacked=True, ax=axes[i])
        axes[i].set_title(f'{col} vs Loan Status (Normalized)')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Proportion')
        axes[i].legend(title='Loan Status', labels=['Rejected', 'Approved'])
        axes[i].tick_params(axis='x', rotation=45)
        
# Hide unused subplots
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 10))
correlation_matrix = df[numerical_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Correlation Matrix of Numerical Features')
plt.show()

# Print highly correlated pairs
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.5:
            high_corr_pairs.append((correlation_matrix.columns[i], 
                                   correlation_matrix.columns[j], 
                                   correlation_matrix.iloc[i, j]))

if high_corr_pairs:
    print("Highly correlated feature pairs (|correlation| > 0.5):")
    for pair in high_corr_pairs:
        print(f"{pair[0]} - {pair[1]}: {pair[2]:.3f}")
else:
    print("No highly correlated feature pairs found (|correlation| > 0.5)")

## Data Preprocessing <a id='preprocessing'></a>

Prepare the data for machine learning by encoding categorical variables and scaling numerical features.

In [ ]:
# Encode categorical variables
le_dict = {}
df_encoded = df.copy()

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    le_dict[col] = le
    
print("Categorical variables encoded:")
for col in categorical_cols:
    print(f"{col}: {dict(zip(le_dict[col].classes_, le_dict[col].transform(le_dict[col].classes_)))}")

In [ ]:
# Prepare features and target
X = df_encoded.drop('loan_status', axis=1)
y = df_encoded['loan_status']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Feature columns: {list(X.columns)}")

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set size: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"Training target distribution: {y_train.value_counts().sort_index()}")
print(f"Test target distribution: {y_test.value_counts().sort_index()}")

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed")

## Model Building <a id='model-building'></a>

Build and train multiple machine learning models to predict loan approval status.

In [ ]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(probability=True, random_state=42)
}

# Train and evaluate models
results = {}

for name, model in models.items():
    print(f"Training {name}...")
    
    # Use scaled data for Logistic Regression and SVM, original for Random Forest
    if name in ['Logistic Regression', 'SVM']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'auc_score': auc_score,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"{name} - Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}\n")

In [ ]:
# Compare model performances
model_comparison = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[model]['accuracy'] for model in results.keys()],
    'AUC Score': [results[model]['auc_score'] for model in results.keys()]
})

print("Model Comparison:")
print(model_comparison)

# Plot model comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].bar(model_comparison['Model'], model_comparison['Accuracy'])
axes[0].set_title('Model Accuracy Comparison')
axes[0].set_ylabel('Accuracy')
axes[0].tick_params(axis='x', rotation=45)

axes[1].bar(model_comparison['Model'], model_comparison['AUC Score'])
axes[1].set_title('Model AUC Score Comparison')
axes[1].set_ylabel('AUC Score')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Model Evaluation <a id='evaluation'></a>

Evaluate the best performing model with detailed metrics and visualizations.

In [ ]:
# Select the best model based on AUC score
best_model_name = max(results, key=lambda x: results[x]['auc_score'])
best_model = results[best_model_name]['model']
best_predictions = results[best_model_name]['predictions']
best_probabilities = results[best_model_name]['probabilities']

print(f"Best Model: {best_model_name}")
print(f"Best AUC Score: {results[best_model_name]['auc_score']:.4f}")
print(f"Best Accuracy: {results[best_model_name]['accuracy']:.4f}")

In [ ]:
# Detailed classification report for the best model
print(f"Detailed Classification Report for {best_model_name}:\n")
print(classification_report(y_test, best_predictions, target_names=['Rejected', 'Approved']))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, best_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Rejected', 'Approved'], 
            yticklabels=['Rejected', 'Approved'])
plt.title(f'Confusion Matrix - {best_model_name}')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# ROC Curve
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_test, best_probabilities)
auc = roc_auc_score(y_test, best_probabilities)

plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'Receiver Operating Characteristic (ROC) Curve - {best_model_name}')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
# Feature Importance (for Random Forest)
if best_model_name == 'Random Forest':
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values(by='importance', ascending=False)
    
    print("Top 10 Most Important Features:")
    print(feature_importance.head(10))
    
    # Plot feature importance
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
    plt.title(f'Top 10 Feature Importances - {best_model_name}')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.show()
else:
    print(f"Feature importance is specific to tree-based models. {best_model_name} does not provide feature importance directly.")

## Insights & Conclusions <a id='insights-conclusions'></a>

Summarize the key findings from our analysis and model performance.

In [ ]:
# Summary of findings
print("=== KEY INSIGHTS FROM THE ANALYSIS ===\n")

print("1. DATASET OVERVIEW:")
print(f"   - Total records: {len(df)}")
print(f"   - Total features: {len(df.columns)-1}")
print(f"   - Target variable distribution: {target_dist[0]} rejected, {target_dist[1]} approved")
print(f"   - Class imbalance ratio: {(target_dist[0]/target_dist[1]):.2f}:1 (rejected:approved)")

print("\n2. DATA QUALITY:")
if missing_values.sum() == 0:
    print("   - No missing values detected in the dataset")
else:
    print(f"   - Missing values detected: {missing_values.sum()}")

print("\n3. FEATURE CHARACTERISTICS:")
print(f"   - Numerical features: {len(numerical_cols)-1} (excluding target)")
print(f"   - Categorical features: {len(categorical_cols)}")

print("\n4. MODEL PERFORMANCE:")
print(f"   - Best performing model: {best_model_name}")
print(f"   - Accuracy achieved: {results[best_model_name]['accuracy']:.4f}")
print(f"   - AUC Score achieved: {results[best_model_name]['auc_score']:.4f}")

print("\n5. BUSINESS IMPLICATIONS:")
print("   - The model can help automate loan approval decisions")
print("   - High AUC score indicates good discriminative ability")
print("   - Feature importance analysis reveals key factors affecting loan approval")

if best_model_name == 'Random Forest':
    print(f"   - Most important factor: {feature_importance.iloc[0]['feature']} (importance: {feature_importance.iloc[0]['importance']:.4f})")

## Summary <a id='summary'></a>

This machine learning project successfully developed a model to predict loan approval status based on applicant characteristics. The dataset contained 45,000 records with various demographic, financial, and loan-specific features.

### Key Achievements:
1. Performed comprehensive exploratory data analysis to understand the dataset
2. Identified patterns between applicant characteristics and loan approval status
3. Built and evaluated three different machine learning models
4. Achieved high accuracy and AUC scores, indicating strong predictive capability
5. Identified the most important features for loan approval decisions

### Recommendations:
1. Deploy the best-performing model to assist with loan approval decisions
2. Regularly monitor model performance and retrain as needed
3. Consider additional features that might improve model performance
4. Implement fairness checks to ensure the model doesn't discriminate against protected groups

The project demonstrates the effectiveness of machine learning in financial decision-making, providing a solid foundation for automated loan approval systems.